# **Introduction**

This notebook is for implementing a Monte-Carlo reinforcement learning method on the Frozen Lake environment offered through a Gymnasium environment. Gymnasium is an open source Python library for developing and comparing reinforcement learning algorithms, through the use of a standardized API. There are four key functions to Gymnasium, namely: ```make()```, ```Env.reset()```, ```Env.step()```, and ```Env.render()```.

As per its [introductory documentation](https://gymnasium.farama.org/introduction/basic_usage/), the core of Gymnasium lies in the high-level Python class ```Env```, which approximately represents a Markov Decision Process (MDP) from reinforcement learning theory. This class allows users of Gymnasium to start new episodes, take actions, and visualize the agent's current state. 

# **Theory on MC Methods**

#### **General RL Theory**:

Reinforcement Learning (RL) is the process of learning how to map situations to actions, such that a numerical **reward signal** is maximized. It is different from other machine learning paradigms in that the learner (or **agent**) is not explictly told which actions to take, but instead must discover which actions yield the most reward by trying them and thus interacting with the **environment**. Similarly, in some instances, actions may affect not only the immediate reward that an agent receives, but also all subsequent rewards. 

These two points make for the most important, distinguishing features of RL, those being **trial and error search** and **delayed reward**.

An RL agent may include one or more of these components, those being:
* A policy - which is the agent's behaviour function, mapping states to actions,
* Value function - how good is it to either be in a state and thereafter follow the policy (**state-value function**) or to be in a state and take an action, thereafter following the policy (**action-value function**), and, 
* A model, which is the agent's perceived representation of the environment. 

Our goal ultimately is to develop a **policy**, which is a mapping between states and actions - effectively telling the agent what action to take in each situation such that it may maximize long-term rewards. We want to use the agent's experience to learn this policy, and it should be such that the reward is maximized. The RL problem is typically split into two separate subsystems, those being **prediction** and **control**.

Prediction is the process of evaluating a given policy to determine how much reward an agent will receive under it, whereas control is the process of improving this policy. These two processes interact simultaneously to perform **policy iteration**, which is the methodology by which we learn an optimal policy.

Utilizing the convention from the 2<sup>nd</sup> edition of Sutton and Barto's *Reinforcement Learning: an Introduction*, the process of  **Generalized Policy Iteration (GPI)** can be used to describe this back and forth interaction. Policy evaluation drives the value function of the policy to be consistent with the current policy, whereas policy improvement makes the policy greedy with respect to the current value function. 

#### **MC Methods**:

The first learning method that will be considered is those of **Monte-Carlo methods**. These are methods of solving the RL problem based on averaging the sample returns achieved over a given episodic trajectory for each state-action pair, where only on the completion of the episode are the value estimates and policies changed. There are multiple states, and the return after taking an action in one state depends on the actions taken in later states in the same episode. Therefore, since all the action selections are undergoing learning, the problem becomes non-stationary from the perspective of an earlier state. For this reason we leverage GPI to tackle the non-stationary behaviour. 

We use MC methods to learn the state-value function for a given policy, taking the place of policy evaluation in GPI. Our goal is therefore to learn $v_{\pi}$, the state-value function, from episodes of experience under policy $\pi$:

$$
S_1, A_1, R_2, \ldots, S_k \sim \pi
$$

Where the *return* is the total discounted reward:

$$
G_t = R_{t+1} + \gamma R_{t+2} + \ldots + \gamma^{T-1}R_T
$$

And the value function is given by the expected reward:

$$
v_{\pi}(s) = \mathbb{E}[G_t ~|~ S_t = s]
$$

MC policy evaluation uses the *empirical mean* return instead of the *expected return*. The algorithm that I will be implementing will utilize an incremental every-visit algorithm, where we update an estimate of $v_{\pi}(s)$, given by $V(s)$, incrementally after an episode $S_1, A_1, R_2, \ldots, S_T$.

For each state $S_t$ with return $G_t$

$$
N(S_t) \leftarrow N(S_t) + 1 \\
\newline
V(S_t) \leftarrow V(S_t) + \frac{1}{N(S_t)}(G_t - V(S_t))
$$

However in non-stationary problems it is often useful to track a running mean:

$$
V(S_t) \leftarrow V(S_t) + \alpha(G_t - V(S_t))
$$

To utilize $V$ as an approximation to $v_{\pi}$, we would require a model of the MDP to determine how we can greedily with respect to $V$ to best improve our policy. Therefore we should do our GPI algorithm with the **action-value function**, not the **state-value function**.

But thus far we have not discussed policy improvement. By using the action-value function, we can improve our policy by greedily selecting actions based on $Q$, our estimate for $q_{\pi}$, as given by:

$$
\pi(s) = \underset{a}{\mathrm{argmax}}~~Q(s,a)
$$

But acting strictly greedily leads to no **exploration**, i.e. we are not truly exploring the entirety of the state and action space. How can we know if there are parts of the state space which may be better than what we have already seen or not? To solve this problem, we can use an $\varepsilon$-greedy methodology of policy improvement, where all $m$ actions are tried with non-zero probability:
* with probability $1-\varepsilon$ choose the greedy action
* with probability $\varepsilon$ choose an action at random. 

$$
\pi(a~|~s) = \left\{
        \begin{array}{ll}
         \varepsilon/m + 1 - \varepsilon \quad if~a^* = \underset{a \in \mathcal{A}}{\mathrm{argmax}}~~Q(s,a) \\ 
         \varepsilon/m \qquad \qquad ~ otherwise
        \end{array}
    \right.
$$

A method of balancing the exploration of new actions and the exploitation of actions which are known to produce desireable rewards is known as **Greedy in the Limit with Infinite Exploration (GLIE)**. The idea behind this is that we sample the $k^{th}$ episode using $\pi$: $\{S_1, A_1, R_2, \ldots, S_T\} \sim \pi$, and for each state $S_t$ and action $A_t$ in the episode:

$$
N(S_t, A_t) \leftarrow N(S_t, A_t) + 1 \\
\newline
Q(S_t, A_t) \leftarrow Q(S_t, A_t) + \frac{1}{N(S_t, A_t)}[G_t - Q(S_t, A_t)]
$$

Which estimates the policy for each state-action pair, and this policy is therefore improved based on the new estimated action-value function $Q$:

$$
\varepsilon \leftarrow 1/k \\
\newline
\pi \leftarrow \varepsilon-greedy(Q)
$$

# **Import Packages**

This section imports the necessary packages.

In [6]:
# inclusions:
import gymnasium as gym
import numpy as np

# **Environment Setup**

This section sets up the environment and defines the relevant functions needed for this implementation. 

In [ ]:
# instantiate the environment:
env = gym.make('FrozenLake-v1', desc = None, map_name = "4x4", is_slippery = True)